In [1]:
%precision

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
CO2_measurement = pd.read_csv('../data/carbosense-raw/CO2_sensor_measurements.csv', sep='\t', index_col=['timestamp'], parse_dates=True)
TH_measurement = pd.read_csv('../data/carbosense-raw/temperature_humidity.csv', sep='\t', index_col=['Timestamp'], parse_dates=True)
sensors_metadata = pd.read_csv('../data/carbosense-raw/sensors_metadata_updated.csv')

In [3]:
grouped_avg_CO2 = CO2_measurement.groupby('SensorUnit_ID').resample(rule='30min').CO2.mean().interpolate()

In [4]:
all_sensors = CO2_measurement.SensorUnit_ID.unique()

In [13]:
%%time
all_frames = pd.DataFrame()
for sensor in all_sensors:
    avg_T = TH_measurement[str(sensor) + '.temperature'].resample(rule = '30min').mean().interpolate()
    avg_H = TH_measurement[str(sensor) + '.humidity'].resample(rule = '30min').mean().interpolate()
    
    LN = CO2_measurement[CO2_measurement['SensorUnit_ID'] == sensor].LocationName[0]
    
    this_frame = pd.DataFrame(columns = ['Sensor_ID', 'T', 'H', 'CO2', 'Location_Name', 'Zone_in_Zurich', 'lon', 'lat', 'altitude'])
    this_frame['T']   = avg_T
    this_frame['H']   = avg_H
    this_frame['CO2'] = grouped_avg_CO2[sensor]
    
    this_frame.loc[:,'Sensor_ID'] = sensor
    this_frame.loc[:,'Location_Name'] = LN
    
    this_frame.loc[:,'Zone_in_Zurich'] = sensors_metadata[sensors_metadata.LocationName == LN].zone.values[0]
    
    all_frames = all_frames.append(this_frame)

CPU times: user 1.13 s, sys: 49.3 ms, total: 1.18 s
Wall time: 1.19 s


In [14]:
# Fill missing values for uniformity between temp hum and co2
all_frames['CO2'] = all_frames['CO2'].interpolate()

In [15]:
all_frames.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 68448 entries, 2017-10-01 00:00:00 to 2017-10-31 23:30:00
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sensor_ID       68448 non-null  int64  
 1   T               68448 non-null  float64
 2   H               68448 non-null  float64
 3   CO2             68448 non-null  float64
 4   Location_Name   68448 non-null  object 
 5   Zone_in_Zurich  68448 non-null  int64  
 6   lon             0 non-null      object 
 7   lat             0 non-null      object 
 8   altitude        0 non-null      object 
dtypes: float64(3), int64(2), object(4)
memory usage: 5.2+ MB


In [16]:
all_frames.to_csv("Carbosense-October2017_curated.csv")

### Exercise : fill missing columns